<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW4/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NLP Homework 4

##Essential packages

##Load data and analysis

##Document classification

###Logistic regression

####Preprocess data<br/>implement the cross validation function

####Train and evaluation section

####Test

###Transformer-based model

####Preprocess data(add token, tokenizaiton, padding)

####Train and evaluation section

####Test

##Token classification

###GRU and LSTM

###Transformer-based model